We start with identifying input parameters for HMM: query id and the paths for MSA input and HMM output.

In [89]:
import subprocess

In [111]:
# Path and input parameters
remote_host = "echidna"
database_path = "/db/uniprot/uniref90.fasta"
q_id = 'Q92835'
hmm_path = '/Users/alina/HMM/hmms/'
align_path = '/Users/alina/HMM/alignments/'

In [119]:
# Build HMM with HMMER
!hmmbuild {hmm_path}{q_id}_blast.hmm {align_path}blast/{q_id}_blast.fasta

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             /Users/alina/HMM/alignments/blast/Q92835_blast.fasta
# output HMM file:                  /Users/alina/HMM/hmms/Q92835_blast.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     Q92835_blast           201  1189  1180     0.52  0.591 

# CPU time: 0.41u 0.00s 00:00:00.41 Elapsed: 00:00:00.41


Here we print the statistics of generated HMMs.
- eff_nseq: effective sequence number.
- M: the match states.
- relent: relative entropy of the match state.
- info: mean information content per match state (not so important).
- p relE: positional relative entropy (not so important).
- compKL: Kullback-Leibler divergence from the average composition.

In [120]:
# Analysis of obtained models
!hmmstat {hmm_path}{q_id}_blast.hmm

# hmmstat :: display summary statistics for a profile file
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
#
# idx  name                 accession        nseq eff_nseq      M relent   info p relE compKL
# ---- -------------------- ------------ -------- -------- ------ ------ ------ ------ ------
1      Q92835_blast         -                 201     0.52   1180   0.59   0.59   0.52   0.01


In [121]:
# Compress the file in a binary output
!hmmpress {hmm_path}{q_id}_blast.hmm

Working...    done.
Pressed and indexed 1 HMMs (1 names).
Models pressed into binary file:   /Users/alina/HMM/hmms/Q92835_blast.hmm.h3m
SSI index for binary model file:   /Users/alina/HMM/hmms/Q92835_blast.hmm.h3i
Profiles (MSV part) pressed into:  /Users/alina/HMM/hmms/Q92835_blast.hmm.h3f
Profiles (remainder) pressed into: /Users/alina/HMM/hmms/Q92835_blast.hmm.h3p


In [115]:
# def get_hmm(hmm_output): # a function retrieving a FASTA sequence from ID
#     result = subprocess.run(['ssh', 'echidna', '/software/packages/hmmer/hmmer-3.3.1/usr/bin/hmmscan', '/db/uniprot/uniref90.fasta', '-entry', hmm_output], stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
#     return result.stdout

In [131]:
# HMM scan
!hmmscan --tblout hmmscan.txt {hmm_path}{q_id}_blast.hmm uniref50_small.fasta

# hmmscan :: search sequence(s) against a profile database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query sequence file:             uniref50_small.fasta
# target HMM database:             /Users/alina/HMM/hmms/Q92835_blast.hmm
# per-seq hits tabular output:     hmmscan.txt
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       UniRef50_A0A5A9P0L4  [L=45354]
Description: peptidylprolyl isomerase n=1 Tax=Triplophysa tibetana TaxID=1572043 RepID=A0A5A9P0L4_9TELE
Scores for complete sequence (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Model    Description
    ------- ------ -----    ------- ------ -----   ---- --  -------- -----------

   [No hits detected that satisf

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Query sequence(s):                         1  (8731 residues searched)
Target model(s):                           1  (1180 nodes)
Passed MSV filter:                         0  (0); expected 0.0 (0.02)
Passed bias filter:                        0  (0); expected 0.0 (0.02)
Passed Vit filter:                         0  (0); expected 0.0 (0.001)
Passed Fwd filter:                         0  (0); expected 0.0 (1e-05)
Initial search space (Z):                  1  [actual number of targets]
Domain search space  (domZ):               0  [number of targets reported over threshold]
# CPU time: 0.00u 0.00s 00:00:00.00 Elapsed: 00:00:00.00
# Mc/sec: 32680.67
//
Query:       UniRef50_A0A821LK81  [L=8733]
Description: VWFA domain-containing protein n=7 Tax=Rotaria TaxID=231623 RepID=A0A821LK81_9BILA
Scores for complete sequence (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Model    Description
    

In [132]:
# program + source + hmm
!hmmsearch --tblout hmmsearch.txt {hmm_path}{q_id}_blast.hmm uniref50_small.fasta

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  /Users/alina/HMM/hmms/Q92835_blast.hmm
# target sequence database:        uniref50_small.fasta
# per-seq hits tabular output:     hmmsearch.txt
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       Q92835_blast  [M=1180]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value  score  bias    exp  N  Sequence Description
    ------- ------ -----    ------- ------ -----   ---- --  -------- -----------

   [No hits detected that satisfy reporting thresholds]


Domain annotation for each sequence (and alignments):

   [No targets detected th